In [1]:
import sqlite3

In [2]:
import pandas as pd # data processing, CSV file

In [3]:
data= pd.read_csv('/Users/isslamkhatir/Downloads/parc2013.csv', sep =';')
data

,ID Hotel,Nom hôtel,Adresse,Code Postal,Ville,Région,Enseigne,Groupe,Chaînes Intégrées / Indépendants,Catégorie,Capacité,Hotel/Résidence,Pays,Monthly 2012,Monthly 2013,Monthly,Daily 2012,Daily 2013,Daily,Unnamed: 19
0,60000,BEST WESTERN Nov Hotel,GJIN BUE SHPATA ROAD,1001,TIRANA,NaN,BEST WESTERN,BEST WESTERN,CI,3.0,10.0,HOTEL,ALBANIE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,47587,SHERATON TIRANA,NaN,NaN,TIRANA,NaN,SHERATON,STARWOOD HOTELS & RESORTS,CI,5.0,151.0,HOTEL,ALBANIE,12.0,3.0,ok,366.0,104.0,ok,NaN
2,35338,ART'OTEL DRESDEN BY PARK PLAZA,OSTRA-ALLEE 33,1067,DRESDEN,Saxe,PARK PLAZA,CARLSON REZIDOR HOTEL GROUP,CI,4.0,174.0,HOTEL,ALLEMAGNE,12.0,3.0,ok,NaN,NaN,NaN,NaN
3,51171,B&B DRESDEN,WEISSERITZSTRASSE 10,1067,DRESDEN,Saxe,B&B,GROUPE B&B HOTELS,CI,2.0,131.0,HOTEL,ALLEMAGNE,12.0,2.0,ok,340.0,93.0,ok,NaN
4,31618,HILTON DRESDEN,AN DER FRAUENKIRCHE 5,1067,DRESDEN,Saxe,HILTON INTERNATIONAL,HILTON WORLDWIDE,CI,4.0,333.0,HOTEL,ALLEMAGNE,12.0,2.0,ok,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14881,61440,BEST WESTERN Sevastopol Hotel,8 NAKHIMOV AVENUE,99011,SEVASTOPOL,NaN,BEST WESTERN,BEST WESTERN,CI,3.0,106.0,HOTEL,UKRAINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14882,46432,RADISSON SAS HOTEL KIEV,YAROSLAVOV VAL STREET 22-24,NaN,KIEV,NaN,RADISSON BLU,CARLSON REZIDOR HOTEL GROUP,CI,4.0,255.0,HOTEL,UKRAINE,12.0,3.0,ok,NaN,NaN,NaN,NaN
14883,61441,Fairmont Grand Hotel Kyiv,"Sky Towers, Peremohy Prospekt",NaN,KIEV,NaN,FAIRMONT HOTELS,RAFFLES INTERNATIONAL LTD,CI,4.0,258.0,HOTEL,UKRAINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14884,61442,SHERATON KIEV PLYMPIYSLY HOTEL,NaN,NaN,KIEV,NaN,SHERATON,STARWOOD HOTELS & RESORTS,CI,NaN,NaN,HOTEL,UKRAINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data = data.rename(columns={'Nom hôtel': 'Nomhotel'})

In [5]:
data.columns = data.columns.str.strip()

In [6]:
connection = sqlite3.connect("parc.db")

In [7]:
data.to_sql('parc2013',connection, if_exists= 'replace')

14886

In [8]:
cursor= connection.cursor()

In [9]:
connection.commit()

In [10]:
sql_query = """
SELECT * FROM parc2013 
"""
rows= cursor.execute(sql_query)

In [11]:
rows

In [12]:
columns = [col[0] for col in rows.description]
columns

['index',
 'ID Hotel',
 'Nomhotel',
 'Adresse',
 'Code Postal',
 'Ville',
 'Région',
 'Enseigne',
 'Groupe',
 'Chaînes Intégrées / Indépendants',
 'Catégorie',
 'Capacité',
 'Hotel/Résidence',
 'Pays',
 'Monthly 2012',
 'Monthly 2013',
 'Monthly',
 'Daily 2012',
 'Daily 2013',
 'Daily',
 'Unnamed: 19']

In [13]:
records = rows.fetchall()

# 1)Combien d'hôtels du groupe Starwood communique leurs résultats mensuels en France ?

In [14]:
sql_query = """
SELECT COUNT(DISTINCT Nomhotel) AS NombreHotels FROM parc2013
WHERE Monthly = 'ok' and Groupe = 'STARWOOD HOTELS & RESORTS'
"""
rows= cursor.execute(sql_query)

In [15]:
counthotel = [col[0] for col in rows.description]
records = rows.fetchall()
nomhotel = pd.DataFrame(records,columns= counthotel)
nomhotel

,NombreHotels
0,142


# 2) Combien d'hôtels ont un ID Hotel supérieur ou égal à 10000 et strictement inférieur à 31490?

In [16]:
sql_query = """
SELECT COUNT(*) AS NombreHotels FROM parc2013
WHERE ('ID Hotel' >= 10000) AND ('ID Hotel' < 31490)
"""
c= cursor.execute(sql_query)

In [17]:
count= [col[0] for col in rows.description]
records = rows.fetchall()
counthotel = pd.DataFrame(records,columns= count)
counthotel

,NombreHotels
0,0


# 3) Quels groupes hôteliers est le plus présent à Amsterdam sur la catégorie 4*, indiquer le nombre d'hôtels et le nombre de chambres?

In [18]:
sql_query = """
SELECT Groupe, COUNT(*) AS nb_hotels, SUM(Capacité) AS nb_rooms FROM parc2013
WHERE Catégorie = 4 AND Pays = 'AMSTERDAM'
ORDER BY nb_hotels,Groupe DESC
"""
c= cursor.execute(sql_query)

In [19]:
grp= [col[0] for col in rows.description]
records = rows.fetchall()
countgroup = pd.DataFrame(records,columns= grp)
countgroup

,Groupe,nb_hotels,nb_rooms
0,None,0,None


# 4) Combien d'hôtels à l'enseigne Mercure sont présents dans les pays suivants (Allemagne, Italie, Pays-Bas, Royaume-Uni et République Tchèque, indiquer le nombre d'hôtels et le nombre de chambres?

In [20]:
sql_query = """
SELECT COUNT(*) AS nb_hotels, SUM(Capacité) AS nb_chambres FROM parc2013
WHERE Enseigne = 'MERCURE HOTEL' AND pays IN ('Allemagne','ALLEMAGNE', 'Italie','ITALIE', 'Pays Bas','PAYS BAS','Royaume Uni','ROYAUME UNI','TCHEQUE REPUBLIQUE')
ORDER BY nb_hotels DESC
"""
c= cursor.execute(sql_query)

In [21]:
htl= [col[0] for col in rows.description]
records = rows.fetchall()
nomhotel = pd.DataFrame(records,columns= htl)
nomhotel

,nb_hotels,nb_chambres
0,225,27637.0


# 5) Quel est le nom d'hôtel qui comporte le plus de caractères au Royaume-Uni, indiquer le nombre de caractères (ex : Ibis Amiens comporte 11 caractères)?

In [22]:
sql_query = """
SELECT Nomhotel, LENGTH(Nomhotel) AS nb_caracteres FROM parc2013
WHERE pays = 'ROYAUME UNI'
ORDER BY nb_caracteres DESC
"""
c= cursor.execute(sql_query)

In [23]:
nom= [col[0] for col in rows.description]
records = rows.fetchall()
records

[('PREMIER INN MANCHESTER CITY CENTRE Central Convention Complex', 61),
 ('PREMIER INN MILTON KEYNES Central South West Furzton Lake', 57),
 ('BEST WESTERN Windmill Village Hotel Golf & Leisure Club', 55),
 ("INKEEPER'S LODGE AYLESBURY (SOUTH), WESTON TURVILLE", 51),
 ('TRAVELODGE WINCHESTER SUTTON SCOTNEY A34 NORTHBOUND', 51),
 ('TRAVELODGE WINCHESTER SUTTON SCOTNEY A34 SOUTHBOUND', 51),
 ('DAYS INN Staffordshire Cannock Norton Canes M6 Toll', 51),
 ('PREMIER INN BIRMINGHAM BROAD STREET Brindley Place', 50),
 ('PREMIER INN BIRMINGHAM CITY CENTRE WATERLOO STREET', 50),
 ('MERCURE HOTEL BRANDON HALL HOTEL AND SPA WARWICKSH', 50),
 ('PREMIER INN Glasgow City Centre Buchanan Galleries', 50),
 ('PREMIER INN MANCHESTER CITY CENTRE Portland Street', 50),
 ('PREMIER INN MANCHESTER CITY CENTRE Deansgate Locks', 50),
 ('PREMIER INN NEWCASTLE CITY CENTRE Millenium Bridge', 50),
 ('PREMIER INN BELFAST CITY CENTRE Cathedral Quarter', 49),
 ("HOLIDAY INN EXPRESS KENT INTERN'L AIRPORT MINSTER", 49),

In [24]:
nomhotel = pd.DataFrame(records,columns= nom)
nomhotel

,Nomhotel,nb_caracteres
0,PREMIER INN MANCHESTER CITY CENTRE Central Con...,61
1,PREMIER INN MILTON KEYNES Central South West F...,57
2,BEST WESTERN Windmill Village Hotel Golf & Lei...,55
3,"INKEEPER'S LODGE AYLESBURY (SOUTH), WESTON TUR...",51
4,TRAVELODGE WINCHESTER SUTTON SCOTNEY A34 NORTH...,51
...,...,...
2677,CREWE HALL,10
2678,THE QUEENS,10
2679,CLARIDGE'S,10
2680,ME London,9


# 6) Combien de noms d'hôtels apparaissent deux ou plusieurs fois dans la colonne "Nom Hôtel"?

In [25]:
sql_query = """
SELECT Nomhotel, COUNT(Nomhotel) AS occurrences
FROM parc2013
GROUP BY Nomhotel
HAVING COUNT(Nomhotel) >= 2;
"""
c= cursor.execute(sql_query)

In [26]:
nom= [col[0] for col in rows.description]
records = rows.fetchall()

In [27]:
nomhotel = pd.DataFrame(records,columns= nom)
nomhotel

,Nomhotel,occurrences
0,BADHOTEL SCHEVENINGEN,2
1,BEST WESTERN ATRIUM HOTEL,2
2,BEST WESTERN CROWN HOTEL,2
3,BEST WESTERN EUROHOTEL,2
4,BEST WESTERN GEORGE HOTEL,3
...,...,...
80,QUALYS HOTEL RUEIL LA DEFENSE,2
81,"RICA VICTORIA HOTEL, LILLEHAMMER",2
82,SOKOS HOTEL SEURAHUONE,2
83,SOKOS HOTEL VAAKUNA,9


In [28]:
connection.close()